In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from string import printable

In [ ]:
#import data
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Daily%20CAB%20Tracker.xlsx"\
    ,filename = 'Daily CAB Tracker.xlsx')
df_compass = pd.read_excel('Daily CAB Tracker.xlsx', sheet_name='Tracker')
df_compass.dropna(axis=0, how='all', subset=['Request ID', 'Customer Name'], inplace=True)

In [ ]:
#SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h3>Hola, %s</h3>
        <p><span style="color: rgb(71, 85, 119);">Parece que tienes varios errores en la informaci&oacute;n registrada en tus cuentas.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Asegurate de revisar el archivo adjunto para validar los campos que est&aacute;n generando conflictos.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Feliz D&iacute;a.</span></p>"""

In [ ]:
def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:  
            if pd.isnull(value) or value != value:
                #Fechas vacias
                value = datetime.strptime('2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else: 
                #Error de formato
                value = datetime.strptime('2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            #Fechas normales
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)
    
    return format_date
    

In [ ]:
# Advisor // Advisor Name // Date Advisor Assigned

Advisor = df_compass[['Request ID', 'RR (internal)', 'Advisor', 'Advisor Name', 'Date Advisor Assigned']].copy()
Advisor['Date Advisor Assigned'] = format_date_column(Advisor['Date Advisor Assigned']) 

for id, row in Advisor.iterrows():
    if row['RR (internal)'] == 'Yes':
        if pd.isnull(row['Advisor']) == True:
            Advisor.loc[id, 'Advisor'] = 'Empty'
        else:
            Advisor.loc[id, 'Advisor'] = None

        if pd.isnull(row['Advisor Name']) == True:
            Advisor.loc[id, 'Advisor Name'] = 'Empty Name'
        else:
            Advisor.loc[id, 'Advisor Name'] = None

        if row['Date Advisor Assigned'] == Timestamp('2030-2-2 00:00:00'):
            Advisor.loc[id, 'Date Advisor Assigned'] = 'Empty date'
        elif row['Date Advisor Assigned'] == Timestamp('2050-2-2 00:00:00'):
            Advisor.loc[id, 'Date Advisor Assigned'] = 'Format Error'
        else:
            Advisor.loc[id, 'Date Advisor Assigned'] = None    
    else:
        Advisor.loc[id, 'Advisor'] = None
        Advisor.loc[id, 'Advisor Name'] = None
        Advisor.loc[id, 'Date Advisor Assigned'] = None

Advisor.dropna(axis=0, how='all', subset=['Advisor', 'Advisor Name', 'Date Advisor Assigned'], inplace=True)
Advisor.drop(axis=1, columns=['RR (internal)'], inplace=True)

In [ ]:
# Date Readaout Done

dat_rdone = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Readout Done']].copy()
dat_rdone['Date Advisor Assigned'] = format_date_column(dat_rdone['Date Advisor Assigned']) 
dat_rdone['Date Readout Done'] = format_date_column(dat_rdone['Date Readout Done']) 

for id, row in dat_rdone.iterrows():
    if row['Date Readout Done'] != Timestamp('2030-2-2 00:00:00') and row['Date Readout Done'] != Timestamp('2050-2-2 00:00:00'):
        if row['Date Advisor Assigned'] != Timestamp('2030-2-2 00:00:00') and row['Date Advisor Assigned'] != Timestamp('2050-2-2 00:00:00'):
            if row['Date Advisor Assigned'] > row['Date Readout Done']:
                dat_rdone.loc[id, 'Date Readout Done'] = 'Date Advisor Assigned greater'
            else:
                dat_rdone.loc[id, 'Date Readout Done'] = None
        else:
            dat_rdone.loc[id, 'Date Readout Done'] = None 
    elif row['Date Readout Done'] == Timestamp('2030-2-2 00:00:00') and row['Date Advisor Assigned'] != Timestamp('2030-2-2 00:00:00'):
        dat_rdone.loc[id, 'Date Readout Done'] = 'Empty Date'
    elif row['Date Readout Done'] == Timestamp('2050-2-2 00:00:00'):
        dat_rdone.loc[id, 'Date Readout Done'] = 'Format error'
    else:
        dat_rdone.loc[id, 'Date Readout Done'] = None

dat_rdone.dropna(axis=0, subset=['Date Readout Done'], inplace=True)
dat_rdone.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)

In [81]:
# Merge

final_report = Advisor.merge(dat_rdone, how='outer', left_on='Request ID', right_on='Request ID')